In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date
import datetime
%matplotlib inline

In [2]:
events = pd.read_csv("/Users/gautamchauhan21/Downloads/push_notification_system_dataset/events_final.csv")

In [3]:
events = events.drop(["Unnamed: 0","transactionid"],axis=1)
print("Total number of visitors present in the dataset: %s" %(len(events.visitor_id.unique())))

Total number of visitors present in the dataset: 29999


In [4]:
# encoding = {"view" : 0, "addtocart" : 1, "transaction" : 2}
events.event.value_counts()

0    1378067
1     689298
2     688736
Name: event, dtype: int64

In [5]:
current_timestamp = 1550009403 * 1000

# add to timestamp
difference = current_timestamp - events.timestamp.max()

# add difference to timestamp
events.timestamp = events.timestamp.map(lambda x : x + difference)

events['timestamp'] = pd.to_datetime(events['timestamp'].astype(int), unit='ms')
events.head()

,timestamp,event,transaction_amt,item_id,visitor_id
0,2018-10-28 00:12:27.329,0,0.000000,167,27815
1,2018-10-28 01:00:29.376,2,10615.425974,2568,4357
2,2018-10-28 00:23:35.039,1,0.000000,3657,27322
3,2018-10-28 00:22:51.126,1,0.000000,3492,28392
4,2018-10-28 00:12:32.318,1,0.000000,1210,18319


In [6]:
# append the datetime format
events['year'] = pd.DatetimeIndex(events['timestamp']).year
events['month'] = pd.DatetimeIndex(events['timestamp']).month
events['day'] = pd.DatetimeIndex(events['timestamp']).day
events['hour'] = pd.DatetimeIndex(events['timestamp']).hour

events.head()

,timestamp,event,transaction_amt,item_id,visitor_id,year,month,day,hour
0,2018-10-28 00:12:27.329,0,0.000000,167,27815,2018,10,28,0
1,2018-10-28 01:00:29.376,2,10615.425974,2568,4357,2018,10,28,1
2,2018-10-28 00:23:35.039,1,0.000000,3657,27322,2018,10,28,0
3,2018-10-28 00:22:51.126,1,0.000000,3492,28392,2018,10,28,0
4,2018-10-28 00:12:32.318,1,0.000000,1210,18319,2018,10,28,0


In [7]:
events['weekday'] = pd.DatetimeIndex(events['timestamp']).weekday

In [8]:
events["platform"] = np.random.choice(["Android", "Ios"], len(events), p=[0.75, 0.25])

In [9]:
events.tail()

,timestamp,event,transaction_amt,item_id,visitor_id,year,month,day,hour,weekday,platform
2756096,2018-12-26 22:23:21.151,0,0.0,6285,13123,2018,12,26,22,2,Ios
2756097,2018-12-26 22:40:28.354,0,0.0,5643,16844,2018,12,26,22,2,Ios
2756098,2018-12-26 22:07:15.739,0,0.0,825,14577,2018,12,26,22,2,Android
2756099,2018-12-26 22:19:05.915,0,0.0,8678,8547,2018,12,26,22,2,Ios
2756100,2018-12-26 22:46:19.126,0,0.0,862,20061,2018,12,26,22,2,Ios


In [10]:
events.to_csv("events_preprocessed.csv",index=False)

# Recency

In [11]:
# events.visitor_id.value_counts().idxmax()
events.visitor_id.mode()

0     443
1    3739
2    8476
dtype: int64

In [12]:
# get the frequency of individual user's activity
user_id = 8476
ev = 2

print("User %s did %s transactions in total" %(user_id,len(events[(events["visitor_id"] == user_id) & (events["event"] == ev)])))

User 8476 did 28 transactions in total


In [13]:
# get latest timestamp for activity

print("His recent transaction was done on: %s"%(events[(events['visitor_id'] == user_id) & (events['event'] == ev)].timestamp.max()))

His recent transaction was done on: 2019-01-27 07:07:58.933000


In [14]:
# get all users who have done transaction 
# all others are given the lowest score since they haven't done any transaction

users_= events[events['event'] == 2].visitor_id.unique()
events_= events[events['event'] == 2]
print("Total % of users who did transactions are:",(len(users_)/len(events_))*100,"%")

Total % of users who did transactions are: 4.355660223946476 %


In [15]:
recency = events.groupby(by='visitor_id', as_index=False)['timestamp'].max()
recency.columns = ['visitor_id','lastpurchasetime']
recency['recency'] = recency['lastpurchasetime'].apply(lambda x: (datetime.datetime.now() - x).days)

# Frequency

In [16]:
#calculate frequency of events
frequency_df = events.groupby(by=['visitor_id'], as_index=False)['event'].count()
frequency_df.columns = ['visitor_id','frequency']
frequency_df.head()

,visitor_id,frequency
0,1,91
1,2,103
2,3,91
3,4,81
4,5,91


# Monetary

In [17]:
def summ(df):
    df = df[["visitor_id", "transaction_amt"]] 
    df = df.groupby('visitor_id')['transaction_amt'].agg('sum').reset_index()
    return df

monetary = summ(events)
monetary.head()

,visitor_id,transaction_amt
0,1,292110.277656
1,2,262238.426654
2,3,108361.800668
3,4,139132.195708
4,5,139624.498537


In [18]:
rfm_df = recency.merge(frequency_df,on='visitor_id')
rfm_df = rfm_df.merge(monetary,on='visitor_id')
rfm_df.head()

,visitor_id,lastpurchasetime,recency,frequency,transaction_amt
0,1,2019-02-09 13:07:42.611,21,91,292110.277656
1,2,2019-02-11 23:37:59.533,19,103,262238.426654
2,3,2019-02-09 19:49:39.239,21,91,108361.800668
3,4,2019-02-11 18:45:20.279,19,81,139132.195708
4,5,2019-02-11 14:07:52.022,19,91,139624.498537


In [19]:
# rfm_df.to_csv("rfm.csv",index=False)

In [22]:
# get recency percentiles :

percentiles = [np.percentile(rfm_df.recency, i) for i in [85, 65, 40, 20]]
R_85, R_65, R_40, R_20 = percentiles[0], percentiles[1], percentiles[2], percentiles[3]

# get frequency percentiles : 
percentiles = [np.percentile(rfm_df.frequency, i) for i in [85, 65, 40, 20]]
F_85, F_65, F_40, F_20 = percentiles[0], percentiles[1], percentiles[2], percentiles[3]

# get monetary percentile : 
percentiles = [np.percentile(rfm_df.transaction_amt, i) for i in [85, 65, 40, 20]]
M_85, M_65, M_40, M_20 = percentiles[0], percentiles[1], percentiles[2], percentiles[3]

(M_85, M_65, M_40, M_20)

(300776.7959688206, 251414.07663127856, 207522.33072516037, 171562.43880574123)

In [23]:
def RScore(val) :
    if val > R_85: return 5
    elif val > R_65: return 4
    elif val > R_40: return 3
    elif val > R_20: return 2
    else : return 1
    
def FScore(val) :
    if val > F_85: return 5
    elif val > F_65: return 4
    elif val > F_40: return 3
    elif val > F_20: return 2
    else : return 1

def MScore(val) :
    if val > M_85: return 5
    elif val > M_65: return 4
    elif val > M_40: return 3
    elif val > M_20: return 2
    else : return 1

In [26]:
rfm_df['recency'] = rfm_df.recency.apply(RScore)
rfm_df['frequency'] = rfm_df.frequency.apply(FScore)
rfm_df['monetary'] = rfm_df.transaction_amt.apply(MScore)
rfm_df.drop('transaction_amt', axis=1, inplace=True)

In [29]:
rfm_df.to_csv("rfm.csv",index=False)